<a href="https://colab.research.google.com/github/Maks3174/Phy/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%BD%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D1%96%D1%86%D1%96%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт модулів

In [9]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

Дані про діабет

In [4]:
# трохи обробки даних
df = pd.read_csv('https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/Healthcare-Diabetes.csv',
                 index_col='Id')

columns = df.columns.delete(-1)
data = df[columns]
df[columns] = (data - data.min()) / (data.max() - data.min())

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Id,,,,,,,,,
1,0.352941,0.743719,0.590164,0.318182,0.000000,0.416873,0.234415,0.483333,1
2,0.058824,0.427136,0.540984,0.263636,0.000000,0.330025,0.116567,0.166667,0
3,0.470588,0.919598,0.524590,0.000000,0.000000,0.289082,0.253629,0.183333,1
4,0.058824,0.447236,0.540984,0.209091,0.111111,0.348635,0.038002,0.000000,0
5,0.000000,0.688442,0.327869,0.318182,0.198582,0.534739,0.943638,0.200000,1


Потрібно на основі інформації пацієнта виявити наявність діабету `Outcome`

# Завдання 1
Створіть клас Dataset для тренувальних та тестових даних

In [15]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

class DiabetesDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.X = data.iloc[:, :-1].values.astype('float32')
        self.y = data.iloc[:, -1].values.astype('float32')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = DiabetesDataset(train_data)
test_dataset = DiabetesDataset(test_data)

# Завдання 2
Створіть DataLoader

In [16]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for X_batch, y_batch in train_loader:
    print(X_batch)
    print(y_batch)
    break

tensor([[0.1176, 0.7236, 0.4754, 0.3000, 0.1596, 0.3921, 0.1469, 0.0667],
        [0.4706, 0.4774, 0.5902, 0.0000, 0.0000, 0.4566, 0.1738, 0.6000],
        [0.2941, 0.8342, 0.6230, 0.0000, 0.0000, 0.5670, 0.1119, 0.1000],
        [0.2353, 0.7136, 0.7049, 0.0000, 0.0000, 0.5459, 0.2421, 0.0167],
        [0.0000, 0.4673, 0.4918, 0.2273, 0.1087, 0.3561, 0.1939, 0.0167],
        [0.1765, 0.6080, 0.4262, 0.0000, 0.0000, 0.4467, 0.0209, 0.0667],
        [0.1176, 0.4523, 0.5574, 0.3818, 0.0000, 0.4739, 0.1815, 0.1000],
        [0.4118, 0.5729, 0.6230, 0.1545, 0.1300, 0.2953, 0.1657, 0.1667],
        [0.1176, 0.7789, 0.4262, 0.2455, 0.6383, 0.4801, 0.0692, 0.0667],
        [0.1765, 0.5578, 0.4590, 0.3545, 0.0000, 0.3734, 0.2045, 0.1500],
        [0.1765, 0.8693, 0.6721, 0.4364, 0.5496, 0.4764, 0.8792, 0.0667],
        [0.4706, 0.5025, 0.6230, 0.0000, 0.0000, 0.4801, 0.0478, 0.3500],
        [0.6471, 0.5578, 0.6885, 0.3636, 0.0000, 0.5806, 0.3617, 0.4000],
        [0.0000, 0.5126, 0.7049, 0.154

# Завдання 3
Створіть нейромережу

In [17]:
class DiabetesNN(nn.Module):
    def __init__(self):
        super(DiabetesNN, self).__init__()
        self.fc1 = nn.Linear(8, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

model = DiabetesNN()
print(model)

DiabetesNN(
  (fc1): Linear(in_features=8, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
)


# Завдання 4
Визначте функцію втрат та оптимізатор

In [18]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Завдання 5
Натренуйте нейромережу. Візміть невелику кількість епох

In [14]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/10], Loss: 0.6562
Epoch [2/10], Loss: 0.6198
Epoch [3/10], Loss: 0.5669
Epoch [4/10], Loss: 0.5102
Epoch [5/10], Loss: 0.4857
Epoch [6/10], Loss: 0.4694
Epoch [7/10], Loss: 0.4672
Epoch [8/10], Loss: 0.4553
Epoch [9/10], Loss: 0.4515
Epoch [10/10], Loss: 0.4456


# Завдання 6
Збережіть нейромережу та зробіть прогноз

In [19]:
torch.save(model.state_dict(), 'diabetes_model.pth')

model.eval()
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch).squeeze()
        predicted = (outputs > 0.5).float()
        print(f"Predicted: {predicted}")
        print(f"Actual: {y_batch}")
        break

Predicted: tensor([0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Actual: tensor([0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
        0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0.])
